# OpenAI Robotics Interaction Demo

In [39]:
import os
import openai
openai.api_type = "azure"
openai.api_base = "https://a9openai.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = "351165ca0090473b8d30b744a507871a"

### Load the robot interface
This is the preamble that will be used to set the context for the call to the codex model. 

These methods are unimplemented, mainly to keep the model input short and to-the-point. Later, this will be swapped for the implemented version to run on the robot.

In [40]:
f = open("robot_interface.py","r")
template = f.read()

print(template)

# Python 3
# Interface for a robot arm

class Color(enum.Enum):
    Blue =   "blue"   #blue
    Green =  "green"  #green
    Yellow = "yellow" #yellow
    Red =    "red"    #red
    Orange = "orange" #orange
    Black =  "black"  #black
    
class SceneObjectType(enum.Enum):
    Block =  "block"  #block
    Bin =    "bin"    #bin
    Ball =   "ball"   #ball
    
class Pose:

class SceneObject:
    def get_color(self) -> Color:
        """Gets the color of the scene object"""
    
    def get_object_type(self) -> SceneObjectType:
        """Gets the object type of the scene object"""
    
    def get_pose(self) -> Pose:
        """Gets the pose of the scene object"""
        
class RobotController:
    def __init__(self):
        """RobotController initializer"""
        
    def get_scene_objects(self) -> List<SceneObject>:
        """Gets all objects in the visual scene."""

    def pick_and_place(self, pick_pose:Pose, place_pose:Pose):
        """The robot will pick up the object at 

### Create your prompt

In [41]:
good_examples =  """

# Pick up the blue block and put it in the red bin.
# The robot can only hold one object at a time.
robot = RobotController()
scene = robot.get_scene_objects()
blue_block = None
red_bin = None
for obj in scene:
    if obj.get_color() == Color.Blue and obj.get_object_type() == SceneObjectType.Block:
        blue_block = obj
    elif obj.get_color() == Color.Red and obj.get_object_type() == SceneObjectType.Bin:
        red_bin = obj
if blue_block != None and red_bin != None:
    robot.pick_and_place(blue_block.get_pose(), red_bin.get_pose())
else:
    print("Could not find blue block or red bin.")

### 

# Pick up all the yellow blocks and put them in the orange bin.
# The robot can only hold one object at a time.
robot = RobotController()
scene = robot.get_scene_objects()

# Find the yellow blocks
yellow_blocks = [b for b in scene if o.get_color() == Color.Yellow and o.get_object_type() == SceneObjectType.Block]

# Find the orange bin
orange_bin = next((o for o in scene if o.get_color() == Color.Orange and o.get_object_type() == SceneObjectType.Bin), None)
        
if yellow_blocks != [] and orange_bin != None:
    for block in yellow_blocks:
        robot.pick_and_place(block.get_pose(), orange_bin.get_pose())
else:
    print("Could not find yellow blocks or orange bin.")

###

"""

In [47]:
prompt = """

# Pick up green block and put it in the black bin.
# The robot can only hold one object at a time.
"""

### Submit the request to OpenAI

In [48]:
response = openai.Completion.create(
  engine="robotics-ai",
  prompt=template+good_examples+prompt,
  temperature=0.25,
  max_tokens=300,
  top_p=0.5,
  frequency_penalty=0,
  presence_penalty=0,
  best_of=1,
  stop=["###", "\n\n"])
response_text = response["choices"][0]["text"]
# print(response)
print(response_text)

robot = RobotController()
scene = robot.get_scene_objects()

# Find the green block
green_block = next((o for o in scene if o.get_color() == Color.Green and o.get_object_type() == SceneObjectType.Block), None)

# Find the black bin
black_bin = next((o for o in scene if o.get_color() == Color.Black and o.get_object_type() == SceneObjectType.Bin), None)
        
if green_block != None and black_bin != None:
    robot.pick_and_place(green_block.get_pose(), black_bin.get_pose())
else:
    print("Could not find green block or black bin.")




### Run the program
First, load up the full program and make sure it looks good.

In [ ]:
f = open("robot_implementation.py","r")
implementation = f.read()

full_code = implementation+prompt+response_text
print(full_code)

Time to run the program!

In [ ]:
full_code = implementation+prompt+response_text
exec(full_code)

# Appendix
### Initialize ROS environment

In [ ]:
!source /opt/ros/noetic/setup.bash
!source ~/catkin_ws/devel/local_setup.sh

Run this in another command window:

    `
    source /opt/ros/noetic/setup.bash
    source ~/catkin_ws/devel/setup.sh
    source ~/catkin_ws/devel/local_setup.sh
    roslaunch niryo_moveit part_3.launch
    `

In [ ]:
! rostopic list

In [ ]:
! rostopic pub /unity/pickup_block std_msgs/String HelloUnity

In [ ]:
import roslibpy
import time

client = roslibpy.Ros(host='localhost', port=8080)
client.run()
print('Is ROS connected?', client.is_connected)

#service = roslibpy.Service(client, '/rosout/get_loggers', 'roscpp/GetLoggers')
#request = roslibpy.ServiceRequest()


#print('Calling service...')
#result = service.call(request)
#print('Service response: {}'.format(result['loggers']))

#talker = roslibpy.Topic(client, '/unity/pickup_block', 'std_msgs/String')

#talker.publish(roslibpy.Message({'data': 'yellow_block'}))
#print('Sending message...')
#time.sleep(1)

#talker = roslibpy.Topic(client, '/unity/pick_and_place', 'niryo_moveit/PickAndPlace')


#talker.unadvertise()
#client.terminate()


In [ ]:
talker.publish(roslibpy.Message({'target': 'blue_block', 'targetPlacement': 'red_bin'}))
print('Sending message...')
time.sleep(1)

In [ ]:
print('Calling service...')
result = service.call(request)
print('Service response: {}'.format(result['loggers']))

In [ ]:
# Python 3
# Interface for a robot arm

import enum
import roslibpy
import time
import os

class Color(enum.Enum):
    Blue =   "blue"   #blue
    Green =  "green"  #green
    Yellow = "yellow" #yellow
    Red =    "red"    #red
    Orange = "orange" #orange
    Black =  "black"  #black
    
class SceneObjectType(enum.Enum):
    Block =  "block"  #block
    Bin =    "bin"    #bin
    Ball =   "ball"   #ball
    
class Pose:
    position: str
    orientation: str

    def __init__(self, json):
        self.position = json["position"]
        self.orientation = json["orientation"]

class SceneObject:
    color: Color
    object_type: SceneObjectType
    pose: Pose
        
    def __init__(self, json):
        self.color = Color(json["color"])
        self.object_type = SceneObjectType(json["object_type"])
        self.pose = Pose(json["pose"])

class RobotController:
    def __init__(self):
        self.ros_client = roslibpy.Ros(host='localhost', port=8080)
        self.ros_client.run()
        print('Is ROS connected?', self.ros_client.is_connected)
        self.talker = roslibpy.Topic(self.ros_client, '/unity/pick_and_place', 'niryo_moveit/PickAndPlace')
        self.service = roslibpy.Service(self.ros_client, '/unity/object_pose_svc', 'niryo_moveit/ObjectPoseService')
        self.get_scene_objects_svc = roslibpy.Service(self.ros_client,
                                                      '/unity/get_scene_objects_svc',
                                                      'niryo_moveit/GetSceneObjectsService')
    
    def get_scene_objects(self):
        """Gets all objects in the visual scene."""
        request = roslibpy.ServiceRequest()
        
        print('Calling service...')
        result = self.get_scene_objects_svc.call(request)
        found_objects = []
        for scene_obj_json in result["scene_objects"]:
            found_objects.append(SceneObject(scene_obj_json))
        
        return found_objects
    
        
    def get_object_pose(self, name):
        """
        Finds the pose of an object matching the specified name.
        """
        request = roslibpy.ServiceRequest()
        request["object_name"] = name

        print('Calling service...')
        result = self.service.call(request)
        
        return result["object_pose"]
    
    def pick_and_place(self, pick_pose, place_pose):
        """The robot will pick up the object at target_pose and move it to the placement_pose"""
        self.talker.publish(roslibpy.Message({'pick_pose': pick_pose, 'place_pose': place_pose}))
        print('Sending message...')
        time.sleep(100)
        
    def __del__(self):
        print('Destructor called. Shutting down ROS connection.')
        #self.talker.unadvertise()
        self.ros_client.terminate()
    
###
        
# Pick up the blue block and put in in the red bin.
# The robot can only grasp one object at a time.
def main():
    robot = RobotController()
    
    scene_objects = robot.get_scene_objects()
    
    # Find blue block and red bin
    blue_block = None
    red_bin = None
    for obj in scene_objects:
        if obj.color == Color.Blue and obj.object_type == SceneObjectType.Block:
            blue_block = obj
        elif obj.color == Color.Red and obj.object_type == SceneObjectType.Bin:
            red_bin = obj
            
    print (str(blue_block))
    print (str(red_bin))
    
    #pick_pose = robot.get_object_pose("blue_block")
    #print ("blue_block pose: ", pick_pose)
    
    #place_pose = robot.get_object_pose("black_bin")
    #print ("black_bin pose: ", place_pose)
    
    #robot.pick_and_place(pick_pose, place_pose)
    
main()
    
###